In [18]:
import numpy as np
import pandas as pd
import matplotlib
import scipy
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import floyd_warshall
from scipy.optimize import minimize_scalar, minimize, root
pd.set_option('display.max_rows', None)

In [19]:
link_char = pd.read_excel('Link_characteristics_ver2.xlsx')
od_pair = pd.read_excel('O_D_matrix_ver2.xlsx')
print(od_pair,'old')
link_char['ca'] = link_char['ca'] * 1000
link_char['o'] = link_char['o'] - 1
link_char['d'] = link_char['d'] - 1
link_char['link'] = link_char['link'] -1
od_matrix = od_pair.values
od_nodes = od_pair.columns - 1
# print(od_matrix.shape[0],'shape 0')
# print(od_matrix.shape[1],'shape 1')
# print(link_data)

      1     2     4     5     10    11    13    14    15    19    20    21  \
0      0  1320  1320  1320  1080  1100  1250   990   950   900   590   590   
1   1320     0  1250  1300  1100  1120   900   950   940  1300   590   680   
2   1320  1250     0  1320  1080  1070   950   900   840   800  1620   640   
3   1320  1300  1320     0  1130   970   910   880   810   730   800   810   
4   1080  1100  1080  1130     0  1330   900   990  1320  1170   950   900   
5   1100  1120  1070   970  1330     0   940  1320  1110   950   740   610   
6   1250   900   950   910   900   940     0   870   860   680   590   620   
7    990   950   900   880   990  1320   870     0  1320  1130   950   870   
8    950   940   840   810  1320  1110   860  1320     0  1320  1270  1140   
9    900  1300   800   730  1170   950   680  1130  1320     0  1320  1110   
10   590   590  1620   800   950   740   590   980  1270  1320     0  1320   
11   590   680   640   810   900   610   620   870  1140  1110  

In [20]:
def tt(df):
    t0 = np.array(df.t0)
    ca = np.array(df.ca)
    x = np.array(df.x)
    return np.multiply(t0,(1+ 0.15*(np.power(np.divide(x,ca),4))))

def convergence(x_n,x_n1):
    return np.sqrt(np.sum(np.power(np.subtract(x_n1,x_n),2))) / np.sum(x_n)

def return_tt(df):
    x_n = np.array(df['x'])
    t0 = np.array(df.t0)
    ca = np.array(df.ca)
    obj =  np.sum(np.multiply(t0,np.add(x_n,0.15*np.divide(np.power(x_n,5),np.power(ca,4)))))
    return obj

def graph_warshall(df):
    n = 24
    graph = np.zeros([n, n])
    for i in range(len(df)):
        o = int(df.iloc[i]['o'])
        d = int(df.iloc[i]['d'])
        graph[o][d] = df.iloc[i]['tt']
    m_graph = np.ma.masked_values(graph,0)
    return csr_matrix(m_graph)
   
def return_alpha(alpha,df):
    y = np.array(df.y)
    x_n = np.array(df.x)
    t0 = np.array(df.t0)
    ca = np.array(df.ca)
    diff=y-x_n
    x_sub = np.add(x_n,alpha * np.subtract(y,x_n))
    obj = np.sum(np.multiply(t0,np.add(diff,(0.15)*diff*np.divide(np.power(x_sub,4),np.power(ca,4)))))
    return obj


In [21]:
## Initialization
link_char['x'] = 0
link_char['y'] = 0
Z = []
T=[]
###
err = 100
step = 1
ca = link_char['ca']
if err > 1e-4:
    print('Step : ', step)
    link_char['y'] = 0
    x_n = np.array(link_char['x'])
    link_char['tt'] = tt(link_char)
    csgraph = graph_warshall(link_char)
    dist,predes = floyd_warshall(csgraph, directed=True, return_predecessors=True)
    #print(pred,'predecessor')## predesseor matrix
    ## Loop for all demands
    for i in range(od_matrix.shape[0]):
        for j in range(od_matrix.shape[1]):
            o = od_nodes[i]
            d = od_nodes[j]
            traffic = od_matrix[i][j]
            if traffic > 0:
                ## Find shortest path using Floyd-Warshal
                path = []
                a = d
                while a != -9999:
                    path.append(a)
                    a = predes[o][a] ## Floyd Warshall predictions
#                 print(path)
                for k in range(len(path)-1):
                    d_n = path[k] 
                    o_n = path[k+1]
                    link_char.loc[(link_char.o==o_n) & (link_char.d==d_n),'y'] +=  traffic
    ## Get alpha
    if step!= 1:
        alpha = minimize_scalar(return_alpha,args=(link_data), bounds=(0.0, 1.0),method='bounded').x
#         plot_alpha(link_data)
#         print(pred1-pred)
    else:
        alpha = 1
        pred1 = predes
    print("Alpha : ",alpha)
    print("Z : ",return_alpha(alpha,link_char))
    Z.append(return_alpha(alpha,link_char))
    y = np.array(link_char['y'])
    x_n1 = x_n + alpha * (y-x_n)
    link_char['x'] = x_n1 ## Update x to update t
    print("T : ",return_tt(link_char))
    T.append(return_tt(link_char))
    if step != 1:
        err= convergence(x_n,x_n1)
        print("Conv : ",err)
    step += 1
link_char['ratio'] = link_char['x'] / link_char['ca']

Step :  1
Alpha :  1
Z :  1318752.1167491213
T :  1195479.6949280372
